## define functions

- contains requered number -> ensure only specific images are pull
- get image -> pull the require and compress image
- scrapp_image_description -> iterate for each of the products
- grucci -> iterate scrap_image

### scrapper

In [4]:
import requests
from PIL import Image
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
from io import BytesIO
import pandas as pd
from openpyxl import load_workbook

from azure.storage.blob import BlobServiceClient, generate_blob_sas, BlobSasPermissions
from azure_key import account_name, account_key, container_name

import psycopg2
from sql_key import dbname, user, password, host, port


In [6]:
def create_soup(url):

    ua = UserAgent()
    header = {'User-Agent':str(ua.chrome)}

    # Send an HTTP request to the URL
    response = requests.get(url, headers = header)
    soup = BeautifulSoup(response.content, 'html.parser')

    return soup


In [5]:
def connect_blob():

    connect_str = 'DefaultEndpointsProtocol=https;AccountName='+account_name+';AccountKey='+account_key+';EndpointSuffix=core.windows.net'

    #use the client to connect
    blob_Service_client = BlobServiceClient.from_connection_string(connect_str)

    #Use the client to connect to the container 
    container_client = blob_Service_client.get_container_client(container_name)

    return container_client

In [6]:
def connect_sql():
    # Database connection parameters
    conn_params = {
        'dbname': dbname[0],
        'user': user[0],
        'password': password[0],
        'host': host[0],
        'port': port
    }
    # Connect to the PostgreSQL server
    conn = psycopg2.connect(**conn_params)
    cursor = conn.cursor()

    return conn, cursor

In [7]:
def images_to_blob(image, image_name,container_client):

    try:
        img_byte_arr = BytesIO()
        image.save(img_byte_arr, format=image.format)
        img_byte_arr = img_byte_arr.getvalue()

        container_client.upload_blob(name= image_name,data=img_byte_arr )
    except Exception as error:
        print(f"Error: {error}")
    

In [8]:
def data_to_SQL(prod_name, prod_description,brand,prod_images_names,cursor,conn):

    try:
        # Insert in Brand_Prod_table
        products_data = [('BrandA', 'Description for BrandA product')]

        insert_products_query = """
        INSERT INTO Products (Brand, Descript) VALUES (%s, %s) RETURNING Brand_Prod_id;
        """
        cursor.execute(insert_products_query)

        # Insert in Image mapping
        insert_characteristics_query = """
        INSERT INTO product_characteristics (Brand_id, Brand, Descript) VALUES (%s, %s, %s);
        """
        for prod_image_name in prod_images_names:
            cursor.execute(insert_characteristics_query, prod_image_name)

        conn.commit()


        print('xxxx')
    except Exception as error:
        print(f"Error: {error}")
        conn.rollback()
    finally:
        cursor.close()
        conn.close()

In [9]:
def contains_required_number(line):
    
    keeping = ['001_100','004_100','005_100']
    # 001: Prod
    # 004: back
    # 005: 3/4 image
    return any(num in line for num in keeping)

In [10]:
def get_image_links(prod_soup):
    
    item_list_1 = prod_soup.find_all(attrs={"data-image-size": "small-retina"})

    ##### get image
    images = []
    i = 0
    for item in item_list_1: 
            
        i+=1 
        if (i == len(item_list_1)) | (i == 1):
            continue
        elif not(contains_required_number(item['srcset'])) :
            continue
        else:
            images = images + [item['srcset']]
        
        i += 1
    
    return images

In [11]:
def get_description(prod_soup):
    
    description_list = prod_soup.find_all(attrs={"class": "product-detail"} )

    #Get description
    description = ''
    
    for a in description_list[0].ul.find_all('li'):
        description = description + ' ' + a.text
        

    return description_list[0].p.text.replace('\n', '').replace('\t', '') + ' ' + description

    

In [12]:
def links_to_images(links,prod_name,container_client):

    ua = UserAgent()
    header = {'User-Agent':str(ua.chrome)}

    i = 0
    for img_link in links:
        response = requests.get('https:' + img_link, headers = header)
        img = Image.open(BytesIO(response.content))

        # Resize the image to 600
        img_resized = img.resize((500, 500))

        img_name = f"Gucci{prod_name}_{i}.jpg"

        # Load to Blob
        #images_to_blob(img, img_name,container_client)
        print(img_name)
        i+=1



In [13]:
def mango(category_url,n_products):

    soup = create_soup(category_url)
    item_list = soup.select('article a')

    
    #container_client = connect_blob()
    container_client = True

    conn, cursor = connect_sql()


    br = 0
    for item in item_list: 
            
        if br == n_products:
            break
        else:

            #Scrapp
            prod_soup = create_soup('https://www.gucci.com' + item['href'])

            prod_images_links = get_image_links(prod_soup=prod_soup)
            prod_description = get_description(prod_soup=prod_soup)

            prod_name = 'HM_'+str(br)  
            
            # Links to image, load to blob and return prod_images_names
            prod_images_names = links_to_images(links = prod_images_links,prod_name= br,container_client=container_client)
            
            #data_to_SQL(prod_name=prod_name, prod_description=prod_description,brand='HM',prod_images_names=prod_images_names,conn=conn)
            print()

        br += 1


## Execute Scrapper

In [33]:
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.edge.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time

url = 'https://shop.mango.com/it/it/c/donna/taglie-plus/giacchette_5fd4b931'

chrome_options = Options()
#chrome_options.add_argument("--headless")  # Ensure GUI is off
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")


with Chrome(options=chrome_options) as browser:
     browser.get(url)
     wait = WebDriverWait(browser, 120)
     
     wait.until(lambda driver: driver.execute_script('return document.readyState') == 'complete')
    
     # Wait for the button to be clickable
     cookie_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.Button_button__qoiWI.CookiesContent_button__2Vj_R')))
     cookie_button.click()

     html = browser.page_source
     


page_soup = BeautifulSoup(html, 'html.parser')

item_list = page_soup.select('div.ProductImage_imageWrapper__dcoT9')
item_list


br = 0
for item in item_list: 

    print(item)
    
    br += 1

br

In [12]:
# Define the URL you want to scrape
category_url = 'https://shop.mango.com/it/it/c/donna/taglie-plus/giacchette_5fd4b931'

#mango(url,3)


soup = create_soup(category_url)
item_list = page_soup.select('div.ProductImage_imageWrapper__dcoT9')
item_list


br = 0
for item in item_list: 

    print(item)
    
    br += 1

br

<div class="ProductImage_productImage__cS5d9"><div class="ProductImage_imageWrapper__dcoT9"><a aria-label="Gilet lino bottoni  - 67069229 - (99)" class="" href="https://shop.mango.com/it/it/p/donna/giacchette/gilet-da-completo/gilet-lino-bottoni_67069229?c=99"><img alt="Gilet lino bottoni  - Dettaglio dell'articolo 5" data-nimg="fill" decoding="async" fetchpriority="high" sizes="(max-width: 1024px) calc(50vw - 0.5px), 511.5px" src="https://shop.mango.com/assets/rcs/pics/static/T6/fotos/S/67069229_99_D5.jpg?ts=1712155008827&amp;im=SmartCrop,width=2048,height=2868&amp;imdensity=1" srcset="https://shop.mango.com/assets/rcs/pics/static/T6/fotos/S/67069229_99_D5.jpg?ts=1712155008827&amp;im=SmartCrop,width=16,height=448&amp;imdensity=1 16w, https://shop.mango.com/assets/rcs/pics/static/T6/fotos/S/67069229_99_D5.jpg?ts=1712155008827&amp;im=SmartCrop,width=20,height=448&amp;imdensity=1 20w, https://shop.mango.com/assets/rcs/pics/static/T6/fotos/S/67069229_99_D5.jpg?ts=1712155008827&amp;im=Smar

8

In [ ]:
<div class="ProductImage_productImage__cS5d9"><div class="ProductImage_imageWrapper__dcoT9"><a class="" href="https://shop.mango.com/it/it/p/donna/giacchette/giacchette/giacca-incrociata-bottone_67090462?c=30" aria-label="Giacca incrociata bottone - 67090462 - (30)"><img alt="Giacca incrociata bottone - Dettaglio dell'articolo 5" loading="lazy" decoding="async" data-nimg="fill" sizes="(max-width: 1024px) calc(50vw - 0.5px), 511.5px" srcset="https://shop.mango.com/assets/rcs/pics/static/T6/fotos/S/67090462_30_D5.jpg?ts=1702458196292&amp;im=SmartCrop,width=16,height=448&amp;imdensity=1 16w, https://shop.mango.com/assets/rcs/pics/static/T6/fotos/S/67090462_30_D5.jpg?ts=1702458196292&amp;im=SmartCrop,width=20,height=448&amp;imdensity=1 20w, https://shop.mango.com/assets/rcs/pics/static/T6/fotos/S/67090462_30_D5.jpg?ts=1702458196292&amp;im=SmartCrop,width=32,height=448&amp;imdensity=1 32w, https://shop.mango.com/assets/rcs/pics/static/T6/fotos/S/67090462_30_D5.jpg?ts=1702458196292&amp;im=SmartCrop,width=48,height=448&amp;imdensity=1 48w, https://shop.mango.com/assets/rcs/pics/static/T6/fotos/S/67090462_30_D5.jpg?ts=1702458196292&amp;im=SmartCrop,width=64,height=448&amp;imdensity=1 64w, https://shop.mango.com/assets/rcs/pics/static/T6/fotos/S/67090462_30_D5.jpg?ts=1702458196292&amp;im=SmartCrop,width=96,height=448&amp;imdensity=1 96w, https://shop.mango.com/assets/rcs/pics/static/T6/fotos/S/67090462_30_D5.jpg?ts=1702458196292&amp;im=SmartCrop,width=128,height=448&amp;imdensity=1 128w, https://shop.mango.com/assets/rcs/pics/static/T6/fotos/S/67090462_30_D5.jpg?ts=1702458196292&amp;im=SmartCrop,width=256,height=448&amp;imdensity=1 256w, https://shop.mango.com/assets/rcs/pics/static/T6/fotos/S/67090462_30_D5.jpg?ts=1702458196292&amp;im=SmartCrop,width=320,height=448&amp;imdensity=1 320w, https://shop.mango.com/assets/rcs/pics/static/T6/fotos/S/67090462_30_D5.jpg?ts=1702458196292&amp;im=SmartCrop,width=368,height=516&amp;imdensity=1 368w, https://shop.mango.com/assets/rcs/pics/static/T6/fotos/S/67090462_30_D5.jpg?ts=1702458196292&amp;im=SmartCrop,width=480,height=672&amp;imdensity=1 480w, https://shop.mango.com/assets/rcs/pics/static/T6/fotos/S/67090462_30_D5.jpg?ts=1702458196292&amp;im=SmartCrop,width=600,height=840&amp;imdensity=1 600w, https://shop.mango.com/assets/rcs/pics/static/T6/fotos/S/67090462_30_D5.jpg?ts=1702458196292&amp;im=SmartCrop,width=640,height=896&amp;imdensity=1 640w, https://shop.mango.com/assets/rcs/pics/static/T6/fotos/S/67090462_30_D5.jpg?ts=1702458196292&amp;im=SmartCrop,width=721,height=1010&amp;imdensity=1 721w, https://shop.mango.com/assets/rcs/pics/static/T6/fotos/S/67090462_30_D5.jpg?ts=1702458196292&amp;im=SmartCrop,width=768,height=1076&amp;imdensity=1 768w, https://shop.mango.com/assets/rcs/pics/static/T6/fotos/S/67090462_30_D5.jpg?ts=1702458196292&amp;im=SmartCrop,width=800,height=1120&amp;imdensity=1 800w, https://shop.mango.com/assets/rcs/pics/static/T6/fotos/S/67090462_30_D5.jpg?ts=1702458196292&amp;im=SmartCrop,width=960,height=1344&amp;imdensity=1 960w, https://shop.mango.com/assets/rcs/pics/static/T6/fotos/S/67090462_30_D5.jpg?ts=1702458196292&amp;im=SmartCrop,width=1024,height=1434&amp;imdensity=1 1024w, https://shop.mango.com/assets/rcs/pics/static/T6/fotos/S/67090462_30_D5.jpg?ts=1702458196292&amp;im=SmartCrop,width=1256,height=1759&amp;imdensity=1 1256w, https://shop.mango.com/assets/rcs/pics/static/T6/fotos/S/67090462_30_D5.jpg?ts=1702458196292&amp;im=SmartCrop,width=1350,height=1890&amp;imdensity=1 1350w, https://shop.mango.com/assets/rcs/pics/static/T6/fotos/S/67090462_30_D5.jpg?ts=1702458196292&amp;im=SmartCrop,width=1656,height=2319&amp;imdensity=1 1656w, https://shop.mango.com/assets/rcs/pics/static/T6/fotos/S/67090462_30_D5.jpg?ts=1702458196292&amp;im=SmartCrop,width=1920,height=2688&amp;imdensity=1 1920w, https://shop.mango.com/assets/rcs/pics/static/T6/fotos/S/67090462_30_D5.jpg?ts=1702458196292&amp;im=SmartCrop,width=2048,height=2868&amp;imdensity=1 2048w" src="https://shop.mango.com/assets/rcs/pics/static/T6/fotos/S/67090462_30_D5.jpg?ts=1702458196292&amp;im=SmartCrop,width=2048,height=2868&amp;imdensity=1" style="position: absolute; height: 100%; width: 100%; inset: 0px; color: transparent;"><span class="classes_srOnly__1Q8Zg">Giacca incrociata bottone Marrone (67090462-CHARLOTT-LM)</span></a></div></div>